In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

def load_and_explore_data(file_path):
    """
    Load data dan eksplorasi awal
    """
    print("=== LOADING DATA ===")
    df = pd.read_csv(file_path)

    print(f"Shape data: {df.shape}")
    print(f"Columns: {list(df.columns)}")
    print("\n=== INFO DATA ===")
    print(df.info())

    print("\n=== STATISTIK DESKRIPTIF ===")
    print(df.describe())

    return df

def check_missing_values(df):
    """
    Cek missing values dan nilai kosong
    """
    print("\n=== MISSING VALUES ===")
    missing_values = df.isnull().sum()
    missing_percent = (missing_values / len(df)) * 100

    missing_df = pd.DataFrame({
        'Column': df.columns,
        'Missing Count': missing_values,
        'Missing Percentage': missing_percent
    })

    print(missing_df[missing_df['Missing Count'] > 0])

    # Cek nilai kosong (empty strings)
    print("\n=== EMPTY VALUES ===")
    for col in df.columns:
        if df[col].dtype == 'object':
            empty_count = (df[col] == '').sum()
            if empty_count > 0:
                print(f"{col}: {empty_count} empty values")

    return missing_df

def check_duplicates(df):
    """
    Cek duplikasi data
    """
    print("\n=== DUPLICATE VALUES ===")

    # Cek duplikasi berdasarkan customerID
    duplicate_ids = df['customerID'].duplicated().sum()
    print(f"Duplicate customer IDs: {duplicate_ids}")

    # Cek duplikasi seluruh row
    duplicate_rows = df.duplicated().sum()
    print(f"Duplicate rows: {duplicate_rows}")

    if duplicate_rows > 0:
        print("Duplicate rows details:")
        print(df[df.duplicated(keep=False)])

    return duplicate_ids, duplicate_rows

def check_data_consistency(df):
    """
    Cek konsistensi data dan nilai yang tidak masuk akal
    """
    print("\n=== DATA CONSISTENCY CHECK ===")

    # Cek nilai unik untuk setiap kolom kategorik
    categorical_cols = df.select_dtypes(include=['object']).columns

    for col in categorical_cols:
        if col != 'customerID':  # Skip customerID karena seharusnya unik
            unique_values = df[col].unique()
            print(f"\n{col}: {unique_values}")

    # Cek range nilai numerik
    print("\n=== NUMERICAL COLUMNS RANGE ===")
    numerical_cols = df.select_dtypes(include=[np.number]).columns

    for col in numerical_cols:
        print(f"{col}: min={df[col].min()}, max={df[col].max()}")

    # Cek nilai yang tidak konsisten
    print("\n=== INCONSISTENT VALUES CHECK ===")

    # Cek TotalCharges (seharusnya numerik tapi mungkin ada string)
    if 'TotalCharges' in df.columns:
        try:
            pd.to_numeric(df['TotalCharges'])
            print("TotalCharges: All values are numeric")
        except:
            non_numeric = df[pd.to_numeric(df['TotalCharges'], errors='coerce').isnull()]
            print(f"TotalCharges: {len(non_numeric)} non-numeric values found")
            print("Non-numeric values:", df['TotalCharges'][pd.to_numeric(df['TotalCharges'], errors='coerce').isnull()].unique())

def clean_data(df):
    """
    Membersihkan data sampah dan nilai yang tidak konsisten
    """
    print("\n=== CLEANING DATA ===")
    df_clean = df.copy()

    # 1. Remove duplikasi
    initial_shape = df_clean.shape[0]
    df_clean = df_clean.drop_duplicates()
    print(f"Removed {initial_shape - df_clean.shape[0]} duplicate rows")

    # 2. Handle customerID duplicates (keep first occurrence)
    df_clean = df_clean.drop_duplicates(subset=['customerID'], keep='first')
    print(f"Removed duplicate customerIDs, final shape: {df_clean.shape}")

    # 3. Clean TotalCharges (convert to numeric, handle spaces)
    if 'TotalCharges' in df_clean.columns:
        # Replace empty strings with NaN
        df_clean['TotalCharges'] = df_clean['TotalCharges'].replace(' ', np.nan)

        # Convert to numeric
        df_clean['TotalCharges'] = pd.to_numeric(df_clean['TotalCharges'], errors='coerce')

        # Handle missing TotalCharges (impute with MonthlyCharges * tenure)
        missing_total = df_clean['TotalCharges'].isnull()
        if missing_total.sum() > 0:
            print(f"Imputing {missing_total.sum()} missing TotalCharges values")
            df_clean.loc[missing_total, 'TotalCharges'] = (
                df_clean.loc[missing_total, 'MonthlyCharges'] *
                df_clean.loc[missing_total, 'tenure']
            )

    # 4. Standardize categorical values
    # Convert Yes/No to consistent format
    yes_no_cols = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']
    for col in yes_no_cols:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].str.strip().str.title()

    # 5. Handle logical inconsistencies
    # Jika PhoneService = No, maka MultipleLines seharusnya 'No phone service'
    if 'PhoneService' in df_clean.columns and 'MultipleLines' in df_clean.columns:
        mask = df_clean['PhoneService'] == 'No'
        df_clean.loc[mask, 'MultipleLines'] = 'No phone service'

    # 6. Remove rows with impossible values
    # Tenure = 0 tapi TotalCharges > 0 (tidak masuk akal)
    if 'tenure' in df_clean.columns and 'TotalCharges' in df_clean.columns:
        impossible_mask = (df_clean['tenure'] == 0) & (df_clean['TotalCharges'] > 0)
        if impossible_mask.sum() > 0:
            print(f"Removing {impossible_mask.sum()} rows with impossible tenure/charges combination")
            df_clean = df_clean[~impossible_mask]

    # 7. Remove rows with negative values (if any)
    numeric_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
    for col in numeric_cols:
        if col in df_clean.columns:
            negative_mask = df_clean[col] < 0
            if negative_mask.sum() > 0:
                print(f"Removing {negative_mask.sum()} rows with negative {col}")
                df_clean = df_clean[~negative_mask]

    print(f"Final cleaned data shape: {df_clean.shape}")
    return df_clean

def handle_outliers(df, method='iqr'):
    """
    Handle outliers dalam data numerik
    """
    print(f"\n=== HANDLING OUTLIERS (method: {method}) ===")
    df_clean = df.copy()

    numeric_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']

    for col in numeric_cols:
        if col in df_clean.columns:
            if method == 'iqr':
                Q1 = df_clean[col].quantile(0.25)
                Q3 = df_clean[col].quantile(0.75)
                IQR = Q3 - Q1
                lower_bound = Q1 - 1.5 * IQR
                upper_bound = Q3 + 1.5 * IQR

                outliers = ((df_clean[col] < lower_bound) | (df_clean[col] > upper_bound))
                outlier_count = outliers.sum()

                print(f"{col}: {outlier_count} outliers detected")
                print(f"  Range: [{lower_bound:.2f}, {upper_bound:.2f}]")

                # Option 1: Remove outliers
                # df_clean = df_clean[~outliers]

                # Option 2: Cap outliers (lebih konservatif)
                df_clean.loc[df_clean[col] < lower_bound, col] = lower_bound
                df_clean.loc[df_clean[col] > upper_bound, col] = upper_bound

                capped_count = ((df_clean[col] == lower_bound) | (df_clean[col] == upper_bound)).sum()
                print(f"  Capped {capped_count} values")

    return df_clean

def create_preprocessing_report(df_original, df_clean):
    """
    Buat laporan preprocessing
    """
    print("\n" + "="*50)
    print("PREPROCESSING REPORT")
    print("="*50)

    print(f"Original data shape: {df_original.shape}")
    print(f"Cleaned data shape: {df_clean.shape}")
    print(f"Rows removed: {df_original.shape[0] - df_clean.shape[0]} ({((df_original.shape[0] - df_clean.shape[0])/df_original.shape[0]*100):.2f}%)")

    print("\n=== MISSING VALUES AFTER CLEANING ===")
    missing_after = df_clean.isnull().sum()
    if missing_after.sum() == 0:
        print("No missing values!")
    else:
        print(missing_after[missing_after > 0])

    print("\n=== DATA TYPES ===")
    print(df_clean.dtypes)

    return df_clean

def main_preprocessing_pipeline(file_path):
    """
    Pipeline utama untuk preprocessing
    """
    print("TELCO CUSTOMER CHURN - DATA PREPROCESSING PIPELINE")
    print("="*60)

    # 1. Load dan explore data
    df = load_and_explore_data(file_path)

    # 2. Check missing values
    missing_info = check_missing_values(df)

    # 3. Check duplicates
    dup_ids, dup_rows = check_duplicates(df)

    # 4. Check data consistency
    check_data_consistency(df)

    # 5. Clean data
    df_clean = clean_data(df)

    # 6. Handle outliers
    df_final = handle_outliers(df_clean, method='iqr')

    # 7. Create report
    df_final = create_preprocessing_report(df, df_final)

    # 8. Save cleaned data
    output_file = file_path.replace('.csv', '_cleaned.csv')
    df_final.to_csv(output_file, index=False)
    print(f"\nCleaned data saved to: {output_file}")

    return df_final

# Contoh penggunaan
if __name__ == "__main__":
    # Ganti dengan path file Anda
    file_path = "dataset/WA_Fn-UseC_-Telco-Customer-Churn.csv"

    # Jalankan preprocessing pipeline
    cleaned_data = main_preprocessing_pipeline(file_path)

    # Tampilkan beberapa baris hasil
    print("\n=== SAMPLE CLEANED DATA ===")
    print(cleaned_data.head())

    print("\n=== PREPROCESSING COMPLETED ===")

TELCO CUSTOMER CHURN - DATA PREPROCESSING PIPELINE
=== LOADING DATA ===
Shape data: (7043, 21)
Columns: ['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn']

=== INFO DATA ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     

In [6]:
import pandas as pd

df = pd.read_csv("dataset/WA_Fn-UseC_-Telco-Customer-Churn_cleaned.csv")
# print(df.isna().sum())  # Melihat jumlah NaN per kolom
print(df['TotalCharges'].unique()[:20])  # tampilkan 20 nilai pertama
print(df['TotalCharges'].dtype)          # cek tipe data
|

SyntaxError: invalid syntax (1051986914.py, line 7)

In [7]:
print(X.isna().sum())  # setelah X dibentuk untuk modeling


NameError: name 'X' is not defined